#### Parameters
- The set of engines that needs maintenance is $M = {1,2,...j}$
- The planning horizon is $T = {1,2,...,t}$
- The available groups are $G = {1,2,..g}$
- The number of working days of  team g on engine j is $\mu_{g,j}$
- Let $s_{g,j,t}$ be the saving of costs if team $g$ starts working on engine $j$ from day $t$

#### Objective function 
$max$  $Z = \sum_{j=1}^{M} \sum_{g=1}^{G} \sum_{t=1}^{T} s_{g,j,t}* X_{g,j,t} $

#### Decision Variable
\begin{equation}
  X_{g,j,t} = \left \{
  \begin{aligned}
    &1, && \text{if team g starts working on engine j at day t}\  \\
    &0, && \text{otherwise}
  \end{aligned} \right.
\end{equation} 

#### Constraints 
##### Maintenance of engine j can be performed at most once during the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}\leq 1$ for all $j \in M$

##### Maintenance must be completed within the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t} * (t + \mu_{g,j} -1) \leq T$ for all $j \in M$

##### Teams can only work on one engine at a time
$\sum_{j=1}^{M} \sum_{t=t_a+1}^{min \{ T,t + \mu_{g,j_a} -1\}} X_{g,j,t} \leq X_{g,j_a,t_a} + (1 - X_{g,j_a,t_a}) * len(M)$ for all $g \in G, t_a \in T, j_a \in M$

##### Max-engine constraint 
$\sum_{j=1}^{M} \sum_{t=1}^{T} X_{g,j,t} \leq K_h^g $ for all  $g \in G$

##### Regional constraint
$\sum_{t=1}^{T} X_{g,j,t} * L_j = \sum_{t=1}^{T} X_{g,j,t} * R_g $ for all  $g \in G, j \in M$

### 1. Linear model

In [1]:
# Global imports
import pandas as pd
from pulp import *
#from pulp import solvers as pulp_solver_classes

In [2]:
df_consultancy = pd.read_csv('../data/RUL_consultancy_predictions_A3.csv', sep=';', index_col='id')

In [3]:
df_prediction = pd.read_excel('../prediction/predictions.xlsx', index_col = 'id')

In [5]:
def create_variables(df, T):
    def get_penalty(g, j, t):
        '''
        Calculates penalty if team g starts working on engine j at day t
        '''
        RUL = df.loc[j].values[0]
        c_per_day = c[j]
        nr_days_costs = t - RUL + mu[g][j] - 1 # CHECK IF THIS IS CORRECT OR SHOULD BE - (RUL + 1)

        if nr_days_costs > 0:
            costs = nr_days_costs * c_per_day
            return costs
        else:
            return 0

    def get_unmaintained_penalty(j, T):
        '''
        Calculates cost of engine j if the maintenance is not done during the planning horizon T 
        '''
        RUL = df.loc[j].values[0]
        c_per_day = c[j]
        nr_days_costs = T - RUL # CHECK IF THIS IS CORRECT OR SHOULD BE - (RUL + 1)

        if nr_days_costs > 0:
            costs = nr_days_costs * c_per_day
            return costs
        else: return 0

    # List with engine IDs
    M = list(df[df['RUL'] <= T].index)
    print(M)

    # Dictionary with team types
    G = {1: 'A', 2: 'A', 3: 'B', 4: 'B'}
    
    # 
    k = {g: 2 if G[g] == 'A' else 2 for g in G.keys()}

    # Dictionary with cost per day for each engine if it is not working
    c = {i:5 for i in range(1,21)}
    c.update({i:7 for i in range(21,41)})
    c.update({i:9 for i in range(41,61)})
    c.update({i:5 for i in range(61,81)})
    c.update({i:3 for i in range(81,101)})

    # Dictionary with maintenance duration for each engine for teams of type A
    mu_a = {i:4 for i in range(1,26)}
    mu_a.update({i:6 for i in range(26,51)})
    mu_a.update({i:3 for i in range(51,76)})
    mu_a.update({i:5 for i in range(76,101)})
    
    # Dictionary with maintenance duration for each engine for teams of type B
    mu_b = {i:mu_a[i]+1 for i in range(1,34)}
    mu_b.update({i:mu_a[i]+2 for i in range(34,68)})
    mu_b.update({i:mu_a[i]+1 for i in range(68,101)})

    # Single look-up dictionary to find the maintenance duration for a team (g) and engine (j)
    mu = {g: {j: mu_a[j] if typ=='A' else mu_b[j] for j in M} for g, typ in G.items()}

    # Dictionary that holds the cost for all possible team, engine, day combinations
    P = {g:{j:{t: get_penalty(g,j,t) for t in range(1, T+1)} for j in M} for g in G}
    
    # Dictionary that holds the cost for an engine if is not maintained during the planning period
    Q = {j: get_unmaintained_penalty(j, T) for j in M}
    
    # 
    total_costs = sum(Q.values())
    
    # Savings
    S = {g:{j:{t: Q[j] - P[g][j][t] if Q[j] - P[g][j][t] > 0 else 0 for t in range(1, T+1)} for j in M} for g in G}
    
    #
    L = {j: 1 if j < 34 else 2 for j in M}
    
    #
    R = {g: 1 if g in [1,3] else 2 for g in G}
    
    return M, G, k, c, mu, P, Q, total_costs, S, L, R

### Pulp Model

In [58]:
def create_model(task, T, M, G, k, c, mu, S, L, R):
    """
    Creates model
    """
    
    # Create the model
    model = LpProblem(name='maintenance-schedule-optimization-2', sense=LpMaximize)

    team_engine_day = [(g, j, t) for g in G for j in M for t in range(1, T+1)]

    # Set LpVariable parameters
    # Natural constraints included
    X = LpVariable.dicts(name='start_day', indexs=team_engine_day, cat='Binary')

    # Add objective function
    model += lpSum(S[g][j][t] * X[(g, j, t)] for g in G for j in M for t in range(1, T+1))
    

    # Add constraints
    # Maintenance for each engine at most once during planning horizon
    for j in M:
        model += lpSum(X[(g, j, t)] for g in G for t in range(1, T+1)) <= 1

    # Maintenance must be completed within the planning period
    for j in M:
        model += lpSum(X[(g, j, t)] * (t + mu[g][j] - 1) for g in G for t in range(1, T+1)) <= T
    
    # Team can only work on one engine at a time
    for g in G:
        for t_a in range(1, T+1):
            for j_a in M:
                model += lpSum(X[(g, j, t)] for j in M for t in range(t_a, min(T, t_a + mu[g][j_a]))) <= (1 * X[(g, j_a, t_a)]) + (1 - X[(g, j_a, t_a)] ) * len(M)
    
#     # Teams can only start on one engine a day
#     for g in G:
#          for t in range(1, T+1):
#                 model += lpSum(X[(g, j, t)] for j in M) <= 1
    
#     # Teams can only work at one engine at a time          
#     for g in G:
#         for t_a in range(1, T+1):
#             for j_a in M:
#                 model += len(M) * X[(g, j_a, t_a)] + lpSum(X[(g, j, t)] \
#                         for t in range(t_a+1, min(t_a + mu[g][j_a],T)) for j in M) <= len(M)
    
    if task >= 2:
        for g in G:
            model += lpSum(X[(g, j, t)] for j in M for t in range(1, T+1)) <= k[g]
    
    if task == 3:
        for g in G:
            for j in M:
                model += lpSum(X[(g, j, t)] for t in range(1, T+1)) * L[j] == lpSum(X[(g, j, t)] for t in range(1, T+1)) * R[g]
            
    
            
    return model

In [50]:
def results_lp(model,total_costs):
    #Print the status of solving
    print("Status = %s" % LpStatus[model.status])
    # Print the value of the objective
    print("Objective = %f" % value(model.objective))

    
    #Print the value of the variables when value > 0 
    for v in model.variables():
        if v.varValue != None and v.varValue > 0:
            print(v.name, "=", v.varValue)
                

    print("Total costs = " , float(total_costs) - value(model.objective))

In [23]:
def main(df,task,T):
    """
    doet alles 
    """
    M, G, k, c, mu, P, Q, total_costs, S, L, R = create_variables(df, T = T)
    model = create_model(task, T, M, G, k, c, mu, S, L, R)
    model.solve()
    
    return results_lp(model,total_costs)

In [ ]:
main(df_consultancy,1,20)

[20, 31, 34, 35, 42, 49, 56, 66, 68, 76, 81, 82]


In [53]:
main(df_consultancy,2,20)

[20, 31, 34, 35, 42, 49, 56, 66, 68, 76, 81, 82]
Status = Optimal
Objective = 489.000000
start_day_(1,_34,_1) = 1.0
start_day_(1,_42,_7) = 1.0
start_day_(2,_20,_7) = 1.0
start_day_(2,_35,_1) = 1.0
start_day_(3,_49,_7) = 1.0
start_day_(3,_76,_1) = 1.0
start_day_(4,_31,_8) = 1.0
start_day_(4,_81,_1) = 1.0
Total costs =  105.0


In [55]:
main(df_consultancy,2,25)

[20, 31, 34, 35, 36, 37, 41, 42, 49, 56, 61, 64, 66, 68, 76, 81, 82, 92, 100]
Status = Optimal
Objective = 760.000000
start_day_(1,_49,_7) = 1.0
start_day_(1,_76,_1) = 1.0
start_day_(2,_34,_1) = 1.0
start_day_(2,_42,_7) = 1.0
start_day_(3,_20,_3) = 1.0
start_day_(3,_31,_8) = 1.0
start_day_(4,_35,_1) = 1.0
start_day_(4,_56,_10) = 1.0
Total costs =  278.0


In [54]:
main(df_consultancy,3,20)

[20, 31, 34, 35, 42, 49, 56, 66, 68, 76, 81, 82]
Status = Optimal
Objective = 363.000000
start_day_(1,_31,_1) = 1.0
start_day_(2,_34,_2) = 1.0
start_day_(2,_42,_8) = 1.0
start_day_(3,_20,_3) = 1.0
start_day_(4,_49,_7) = 1.0
start_day_(4,_76,_1) = 1.0
Total costs =  231.0


## Optimization Task 2

#### Parameters
- The set of engines that needs maintenance is $M = {1,2,...j}$
- The planning horizon is $T = {1,2,...,t}$
- The available groups are $G = {1,2,..g}$
- The number of working days of  team g on engine j is $\mu_{g,j}$
- Let $P_{g,j,t}$ be the penalty if team $g$ starts working on engine $j$ from day $t$
- Let $Q_j$ be the cost of engine $j$ if the maintenance is not done during the planning horizon $T$ 

#### Objective function 
$min$  $Z = \sum_{j=1}^{M} \sum_{g=1}^{G} \sum_{t=1}^{T} P_{g,j,t}* X_{g,j,t} + \sum_{j=1}^{M} (( 1 - \sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}) * Q_j)$

#### Decision Variable
\begin{equation}
  X_{g,j,t} = \left \{
  \begin{aligned}
    &1, && \text{if team g starts working on engine j at day t}\  \\
    &0, && \text{otherwise}
  \end{aligned} \right.
\end{equation} 

#### Constraints 
##### Maintenance of engine j can be performed at most once during the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}\leq 1$ for all $j \in M$

##### Maintenance must be completed within the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t} * (t + \mu_{g,j} -1) \leq T$ for all $j \in M$

##### Teams can only work on one engine at a time
$\sum_{j=1}^{M} \sum_{t=t_a+1}^{min \{ T,t + \mu_{g,j_a} -1\}} X_{g,j,t} \leq X_{g,j_a,t_a} + (1 - X_{g,j_a,t_a}) * len(M)$ for all $g \in G, t_a \in T, j_a \in M$

##### Max-engine constraint 
$\sum_{j=1}^{M} \sum_{t=1}^{T} X_{g,j,t} \leq K_h^g $ for all  $g \in G$